In [1]:
# Generating a toy dataset.
# DO NOT MODIFY THIS PART

import numpy as np
import math
import random as rand

paras = list((rand.random() - 0.5 for _ in range(13)))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def y_gen(x):
    h_11 = sigmoid(paras[0] * x[0] + paras[1] * x[1] + paras[2])
    h_12 = sigmoid(paras[3] * x[0] + paras[4] * x[1] + paras[5])
    h_13 = sigmoid(paras[6] * x[0] + paras[7] * x[1] + paras[8])
    h_21 = sigmoid(paras[9] * h_11 + paras[10] * h_12 + paras[11] * h_13 + paras[12])
    return h_21 + ((rand.random()-0.5)/100 if rand.random()>0.6 else 0)

n = 300
x = list(zip((rand.random() - 0.5 for _ in range(n)), (rand.random() - 0.5 for _ in range(n))))
y = list(map(y_gen, x))
y = [(i-min(y))/(max(y)-min(y)) for i in y]

from sklearn.model_selection import train_test_split

r = 0.2
x_train_valid, x_test, y_train_valid, y_test = train_test_split(x, y, test_size=r)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=r)

In [2]:
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [3]:
# Initialize parameters
# Initialize parameters
input_size = 2
hidden_size = 3
output_size = 1
learning_rate = 0.01
epochs = 10000

# Seed for reproducibility
np.random.seed(42)

# Initialize weights and biases
W1 = np.random.randn(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size)
b2 = np.zeros((1, output_size))

# Training loop
for epoch in range(epochs):
    # Shuffle the training data each epoch
    permutation = np.random.permutation(len(x_train))
    x_train_shuffled = np.array(x_train)[permutation]
    y_train_shuffled = np.array(y_train)[permutation]

    # Batch gradient descent
    for i in range(len(x_train_shuffled)):
        X = x_train_shuffled[i].reshape(1, 2)
        y_true = y_train_shuffled[i].reshape(1, 1)

        # Forward pass
        z1 = np.dot(X, W1) + b1
        a1 = sigmoid(z1)
        z2 = np.dot(a1, W2) + b2
        a2 = sigmoid(z2)

        # Compute loss
        loss = np.mean((y_true - a2)**2) / 2

        # Backward pass
        dz2 = a2 - y_true
        dW2 = np.dot(a1.T, dz2)
        db2 = np.sum(dz2, axis=0, keepdims=True)
        dz1 = np.dot(dz2, W2.T) * sigmoid_derivative(z1)
        dW1 = np.dot(X.T, dz1)
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # Update weights and biases
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2

    # Print loss every 1000 epochs
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

# Function to predict output
def predict(X):
    X = np.array(X).reshape(1, 2)  # Reshape in case single example is passed
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    return np.round(a2)

# Testing on validation set
predictions = [predict(x) for x in x_valid]
print("Predictions on validation set:")
print(predictions)
test_predictions = predict(x_test)

# Function to calculate accuracy
def calculate_accuracy(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    correct = np.sum(y_true == y_pred)
    accuracy = correct / len(y_true)
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(y_test, test_predictions.flatten())
print("Accuracy on test set:", accuracy)

Epoch 0, Loss: 0.005145603257004746
Epoch 1000, Loss: 0.00023041782023585494
Epoch 2000, Loss: 0.0024927307271440952
Epoch 3000, Loss: 0.011844876224569107
Epoch 4000, Loss: 1.115562976595621e-05
Epoch 5000, Loss: 7.914297021988937e-07
Epoch 6000, Loss: 0.00011023378685322629
Epoch 7000, Loss: 0.0013433650152332988
Epoch 8000, Loss: 0.026557228539593802
Epoch 9000, Loss: 0.00042193997919596365
Predictions on validation set:
[array([[1.]]), array([[0.]]), array([[1.]]), array([[1.]]), array([[0.]]), array([[1.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[1.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[1.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[1.]]), a

ValueError: cannot reshape array of size 120 into shape (1,2)